# UCU - Proyecto Webscraping
Proyecto para la Licenciatura en Datos y Negocios de la Universidad Católica del Uruguay

# Scraping de propiedades — Inmobiliaria Uruguay
Este notebook descarga listados **públicos** desde *inmobiliariauruguay.com* usando `requests`
y analiza el HTML con `BeautifulSoup` para extraer:

- Ciudad (agrupado por **Departamento** en el sitio, p. ej. *Montevideo*, *Maldonado*, *Canelones*)
- Precio (USD)
- Tamaño (m², si está disponible)
- Dormitorios (si está disponible)
- Link al aviso

Requisitos:
- `requests`, `beautifulsoup4` (instalar con `pip install requests beautifulsoup4`)

Comentario:
- Si bien se utilizaron las herramientas sugeridas por la consigna, también se aplica el uso de otras que fueron aprendidas en videos y tutoriales visualizados para obtener un mejor resultado

## 1) Imports y funciones auxiliares

En este bloque se definen las librerías necesarias y funciones de apoyo para realizar el web scraping en el sitio **inmobiliariauruguay.com**.

- **Imports:** Se cargan librerías como `re`, `json`, `time`, `requests` y `BeautifulSoup` para manejar expresiones regulares, trabajar con JSON, controlar tiempos, hacer peticiones HTTP y parsear el HTML.
- **HEADERS y BASE:** Se configura un encabezado de navegador (User-Agent) para evitar bloqueos y se define la URL base del sitio.
- **Sesión de requests:** Se crea una sesión HTTP con cabeceras personalizadas para reutilizar la conexión.
- **Función `get_soup(url)`:** Realiza la petición al sitio, valida la respuesta y devuelve el HTML en formato `BeautifulSoup` para su análisis.
- **Función `to_int(text)`:** Extrae números de un texto, eliminando puntos de miles y convirtiéndolos a enteros.
- **Función `clean_m2(x)`:** Limpia textos de áreas eliminando `m2` o `m²`, y los transforma a enteros usando `to_int`.
- **Función `parse_title_city(title)`:** A partir del título de un aviso, intenta separar el nombre de la ciudad (última parte luego de coma o guion).


In [5]:
import re
import json
import time
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Accept-Language": "es-ES,es;q=0.9"
}

BASE = "https://inmobiliariauruguay.com"

session = requests.Session()
session.headers.update(HEADERS)

def get_soup(url):
    resp = session.get(url, timeout=30)
    resp.raise_for_status()
    return BeautifulSoup(resp.text, "html.parser")

def to_int(text):
    if text is None:
        return None
    nums = re.findall(r"[0-9][0-9\.]*", text.replace("\xa0"," "))
    if not nums:
        return None
    n = nums[0].replace(".", "")
    try:
        return int(n)
    except ValueError:
        return None

def clean_m2(x):
    if x is None:
        return None
    x = x.replace("m2", "").replace("m²", "")
    return to_int(x)

def parse_title_city(title):
    if not title:
        return None
    import re
    parts = re.split(r"[,–-]", title)
    last = parts[-1].strip() if parts else None
    if last:
        last = re.sub(r"\.$", "", last)
    return last or None

## 2) Listado y paginación

La función **`extract_cards_from_listing`** se encarga de recorrer los listados de propiedades y extraer la información principal de cada aviso.

- **Parámetros:**
  - `list_url`: URL inicial del listado de propiedades.
  - `max_pages`: número máximo de páginas a recorrer (por defecto 5).

- **Flujo de trabajo:**
  1. Crea una lista `out` donde se guardarán los resultados.
  2. Descarga y procesa la página con `get_soup`.
  3. Busca los enlaces de propiedades dentro de etiquetas `h2 a` o `h3 a`.
  4. Para cada tarjeta:
     - Obtiene el **título** de la propiedad.
     - Intenta encontrar el **precio** buscando hacia arriba en el árbol HTML hasta detectar un texto con `"USD"`.
     - Construye un diccionario con `title`, `price_text` y la URL absoluta del detalle.
  5. Verifica si existe un enlace de paginación con el número de la siguiente página (`page+1`). Si lo encuentra, actualiza `url` y continúa el bucle.
  6. El proceso se repite hasta llegar al límite de páginas o cuando no hay más resultados.

- **Salida:**
  Devuelve una lista de diccionarios con los datos de cada propiedad encontrada en el listado.


In [6]:
def extract_cards_from_listing(list_url, max_pages=5):
    out = []
    page = 1
    url = list_url
    while page <= max_pages and url:
        soup = get_soup(url)
        for card in soup.select("h2 a, h3 a"):
            href = card.get("href")
            if not href:
                continue
            title = card.get_text(strip=True)
            price_el = None
            candidate = card
            for _ in range(5):
                candidate = candidate.find_parent()
                if not candidate: break
                price_el = candidate.find(string=lambda s: isinstance(s, str) and "USD" in s)
                if price_el: break
            price_text = (price_el.strip() if isinstance(price_el, str) else price_el.get_text(strip=True)) if price_el else None
            out.append({
                "title": title,
                "price_text": price_text,
                "detail_url": urljoin(BASE, href)
            })
        next_link = None
        for a in soup.select("a"):
            if a.get_text(strip=True) == str(page+1):
                next_link = urljoin(BASE, a.get("href"))
        if next_link and next_link != url:
            url = next_link
            page += 1
        else:
            break
    return out

## 3) Parser de la página de detalle

La función **`parse_detail`** extrae la información detallada de una propiedad a partir de su página individual.

- **Parámetros:**
  - `detail_url`: enlace a la página de detalle de la propiedad.

- **Flujo de trabajo:**
  1. Descarga el HTML con `get_soup` y localiza el **título** en un `<h1>` o `<h2>`.
  2. Inicializa variables para almacenar: **precio**, **superficie (m²)**, **número de dormitorios** y **ciudad**.
  3. Recorre todos los elementos `<li>` de la página, creando un diccionario `detalles` con clave/valor a partir de textos del tipo `"Etiqueta: Valor"`.
  4. Busca dentro de `detalles`:
     - Claves relacionadas con **precio** (`"precio"`) y las convierte en entero.
     - Claves de **tamaño/superficie** (e.g. `"superficie"`, `"área de terreno"`, `"total edificado"`) y las limpia con `clean_m2`.
     - Claves de **habitaciones** (`"dormitorios"`, `"habitaciones"`) y las transforma en entero.
  5. Intenta obtener la **ciudad** desde la clave `"departamento"` del diccionario; si no está, la deduce a partir del título con `parse_title_city`.
  6. Si el precio aún no fue encontrado, busca directamente en el HTML un texto con el patrón `"USD xxxx"`.
  7. Devuelve un diccionario con los datos estandarizados:
     - `ciudad`
     - `precio`
     - `tamano` (m²)
     - `habitaciones`
     - `link` (URL del detalle)


In [8]:
def parse_detail(detail_url):
    soup = get_soup(detail_url)
    title_el = soup.find(["h1","h2"])
    title = title_el.get_text(strip=True) if title_el else None

    price = None
    size_m2 = None
    bedrooms = None
    city = None

    detalles = {}
    for li in soup.select("li"):
        txt = li.get_text(" ", strip=True)
        if ":" in txt:
            k, v = [t.strip() for t in txt.split(":", 1)]
            detalles[k.lower()] = v

    for key in ["precio"]:
        if key in detalles:
            price = to_int(detalles[key])

    for key in ["tamaño de inmueble", "tamaño del inmueble", "superficie", "area de terreno", "área de terreno", "total edificado", "dificada"]:
        if key in detalles and size_m2 is None:
            size_m2 = clean_m2(detalles[key])

    for key in ["dormitorios", "dormitorio", "habitaciones"]:
        if key in detalles and bedrooms is None:
            bedrooms = to_int(detalles[key])

    if "departamento" in detalles:
        city = detalles["departamento"].split(",")[0].strip()
    if not city and title:
        city = parse_title_city(title)

    if price is None:
        import re
        usd_text = soup.find(string=re.compile(r"USD\s*[0-9\.,]+"))
        if usd_text:
            price = to_int(str(usd_text))

    return {
        "ciudad": city,
        "precio": price,
        "tamano": size_m2,
        "habitaciones": bedrooms,
        "link": detail_url
    }

## 4) Ejecución principal y guardado a `propiedades.json`

La función **`scrape_por_ciudades`** orquesta el scraping por ciudad, combinando el listado y el detalle de cada propiedad, y guarda el resultado en un JSON.

- **Parámetros:**
  - `objetivos`: diccionario `{nombre_ciudad: url_listado}` a recorrer.
  - `min_props`: mínimo de propiedades por ciudad a recopilar (por defecto 10).
  - `max_pages`: máximo de páginas de listado a navegar por ciudad.
  - `delay`: pausa (en segundos) entre requests de detalle para ser respetuosos con el sitio.

- **Flujo de trabajo:**
  1. Inicializa `data = {"ciudades": []}` para acumular resultados.
  2. Itera sobre cada ciudad en `objetivos`:
     - Llama a **`extract_cards_from_listing`** para obtener las “tarjetas” (título, precio textual si se encuentra y `detail_url`) desde la URL del listado, respetando `max_pages`.
     - Recorre esas tarjetas y, evitando duplicados con `seen`, visita cada `detail_url`.
     - En cada detalle, llama a **`parse_detail`** para extraer información estandarizada (`precio`, `tamano` en m², `habitaciones`, `link`, y opcionalmente `ciudad`).
     - Si la página de detalle no expone la ciudad, la completa con el nombre del bloque (`nombre`).
     - Agrega la propiedad al arreglo `props` y duerme `delay` segundos entre requests.
     - Maneja errores por propiedad sin detener el scraping del resto (muestra el error y continúa).
  3. Agrega al JSON final un bloque por ciudad con la forma:
     ```json
     {
       "nombre": "<Ciudad>",
       "propiedades": [ /* hasta min_props objetos */ ]
     }
     ```
  4. Devuelve `data` con todas las ciudades.

In [9]:
def scrape_por_ciudades(objetivos, min_props=10, max_pages=5, delay=1.0):
    data = {"ciudades": []}
    import time
    for nombre, list_url in objetivos.items():
        print(f"Recolectando en {nombre} ...")
        cards = extract_cards_from_listing(list_url, max_pages=max_pages)
        print(f"  {len(cards)} tarjetas encontradas")
        props = []
        seen = set()
        for card in cards:
            if len(props) >= min_props:
                break
            detail = card["detail_url"]
            if detail in seen:
                continue
            seen.add(detail)
            try:
                info = parse_detail(detail)
                if not info.get("ciudad"):
                    info["ciudad"] = nombre
                props.append({
                    "precio": info.get("precio"),
                    "tamano": info.get("tamano"),
                    "habitaciones": info.get("habitaciones"),
                    "link": info.get("link")
                })
                print(f"    OK: {detail}")
                time.sleep(delay)
            except Exception as e:
                print(f"    Error {detail}: {e}")
        data["ciudades"].append({
            "nombre": nombre,
            "propiedades": props[:min_props]
        })
    return data

OBJETIVOS = {
    "Montevideo": "https://inmobiliariauruguay.com/search-results/?type%5B%5D=urbana&states%5B%5D=montevideo",
    "Maldonado": "https://inmobiliariauruguay.com/state/maldonado/",
    "Canelones": "https://inmobiliariauruguay.com/state/canelones/",
}

data = scrape_por_ciudades(OBJETIVOS, min_props=10, max_pages=5, delay=0.5)

with open("propiedades.json", "w", encoding="utf-8") as f:
    import json
    json.dump(data, f, ensure_ascii=False, indent=2)

print("Archivo generado: propiedades.json")

Recolectando en Montevideo ...
  22 tarjetas encontradas
    OK: https://inmobiliariauruguay.com/property/apartamento-de-2-dormitorios-en-buceo-montevideo/
    OK: https://inmobiliariauruguay.com/property/casa-2-dormitorios-fondo-entrada-auto-con-porton-en-la-union-montevideo/
    OK: https://inmobiliariauruguay.com/property/2-casa-en-venta-en-punta-de-manga-montevideo/
    OK: https://inmobiliariauruguay.com/property/2-casas-en-un-mismo-padron-en-puntas-de-manga-montevideo/
    OK: https://inmobiliariauruguay.com/property/galpon-a-la-venta-en-montevideo-2/
    OK: https://inmobiliariauruguay.com/property/galpon-a-la-venta-en-montevideo/
    OK: https://inmobiliariauruguay.com/property/apartamento-en-montevideo-9/
    OK: https://inmobiliariauruguay.com/property/apartamento-en-montevideo-8/
    OK: https://inmobiliariauruguay.com/property/hermoso-apartamento-en-montevideo/
    OK: https://inmobiliariauruguay.com/property/apartamento-en-montevideo-6/
Recolectando en Maldonado ...
  45 t